In [4]:
import transformers 
import torch
import math
import random

tokenizer = transformers.AutoTokenizer.from_pretrained("roberta-base")
model = transformers.RobertaForMaskedLM.from_pretrained("roberta-base")

inputs = tokenizer("I am so <mask>", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# retrieve index of <mask>
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]

predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
tokenizer.decode(predicted_token_id)

labels = tokenizer("I am so <mask>", return_tensors="pt")["input_ids"]
# mask labels of non-<mask> tokens
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

print(labels)
outputs = model(**inputs, labels=labels)
outputs.loss.item()

c:\Users\liore\miniconda3\envs\nlp_kernel\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[ -100,  -100,  -100,  -100, 50264,  -100]])


15.918440818786621

In [5]:
text = "I am so <mask>"
top_k = 5
input_ids = tokenizer.encode(text, return_tensors="pt")

# Get the logits for masked positions
logits = model(input_ids)["logits"]

# Find the position of the '<mask>' token in the input
mask_token_index = torch.where(input_ids == 524)[1].item()
print(tokenizer.mask_token_id)
print(mask_token_index)
# Get the probabilities for the top-k predictions
top_k_values, top_k_indices = torch.topk(logits[0, mask_token_index], top_k)
top_k_probabilities = torch.nn.functional.softmax(top_k_values, dim=-1)

# Convert indices back to tokens
predicted_tokens = tokenizer.convert_ids_to_tokens(top_k_indices.tolist())

predicted_tokens, top_k_probabilities.tolist()

50264
2


(['Ġam', 'Ġis', "'m", 'Ġwas', 'Ġfeel'],
 [0.9999134540557861,
  3.9379392546834424e-05,
  2.9938039006083272e-05,
  8.689116839377675e-06,
  8.550994607503526e-06])

In [6]:
import torch
from transformers import RobertaTokenizer, RobertaModel, RobertaForMaskedLM
from sklearn.metrics.pairwise import cosine_similarity

model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
roberta_model = RobertaModel.from_pretrained(model_name)
masked_language_model = RobertaForMaskedLM.from_pretrained(model_name)

###############1.1###############
def get_word_vector(word_index, tokenized_sentence, model):
    outputs = model(**tokenized_sentence)
    last_hidden_states = outputs.last_hidden_state
    word_vector = last_hidden_states[0, word_index]
    return word_vector

###############2###############
similar_sentence_1 = 'I love you'
similar_sentence_2 = 'I love him'
tokenized_similar_sentence_1 = tokenizer(similar_sentence_1, return_tensors='pt')
tokenized_similar_sentence_2 = tokenizer(similar_sentence_2, return_tensors='pt')

love_1_index = tokenized_similar_sentence_1["input_ids"][0].tolist().index(tokenizer.encode(" love")[1])
love_2_index = tokenized_similar_sentence_2["input_ids"][0].tolist().index(tokenizer.encode(" love")[1])
love_1_vector = get_word_vector(love_1_index, tokenized_similar_sentence_1, roberta_model)
love_2_vector = get_word_vector(love_2_index, tokenized_similar_sentence_2, roberta_model)

print('love similarity')
print(cosine_similarity(love_1_vector.detach().numpy().reshape(1, -1), love_2_vector.detach().numpy().reshape(1, -1)))
###############3###############
different_sentence_1 = 'She decided to clip her hair back with a colorful barrette.'
different_sentence_2 = 'Some have come out of nowhere, others have taken months to catch on, and all of them could become ubiquitous in the blink of a TikTok clip.'
tokenized_different_sentence_1 = tokenizer(different_sentence_1, return_tensors='pt')
tokenized_different_sentence_2 = tokenizer(different_sentence_2, return_tensors='pt')

bass_1_index = tokenized_different_sentence_1["input_ids"][0].tolist().index(tokenizer.encode(" clip")[1])
bass_2_index = tokenized_different_sentence_2["input_ids"][0].tolist().index(tokenizer.encode(" clip")[1])
bass_1_vector = get_word_vector(bass_1_index, tokenized_different_sentence_1, roberta_model)
bass_2_vector = get_word_vector(bass_2_index, tokenized_different_sentence_2, roberta_model)

print('bass similarity')
print(cosine_similarity(bass_1_vector.detach().numpy().reshape(1, -1), bass_2_vector.detach().numpy().reshape(1, -1)))

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


love similarity
[[0.9897104]]
bass similarity
[[0.86344]]


In [7]:
###############3###############
different_sentence_1 = 'The teacher asked Sarah to lead the class discussion.'
different_sentence_2 = 'Be careful not to touch that pencil; it has a lead tip.'
tokenized_different_sentence_1 = tokenizer(different_sentence_1, return_tensors='pt')
tokenized_different_sentence_2 = tokenizer(different_sentence_2, return_tensors='pt')

bass_1_index = tokenized_different_sentence_1["input_ids"][0].tolist().index(tokenizer.encode(" lead")[1])
bass_2_index = tokenized_different_sentence_2["input_ids"][0].tolist().index(tokenizer.encode(" lead")[1])
bass_1_vector = get_word_vector(bass_1_index, tokenized_different_sentence_1, roberta_model)
bass_2_vector = get_word_vector(bass_2_index, tokenized_different_sentence_2, roberta_model)

print('bass similarity')
print(cosine_similarity(bass_1_vector.detach().numpy().reshape(1, -1), bass_2_vector.detach().numpy().reshape(1, -1)))

bass similarity
[[0.8710439]]


In [27]:
from collections import Counter
import utils
#Returns 2 values: Article dicts: containing the Topic and the article text
def get_file_data(file_name, lower = False):
    #Each topic is separated by header line, empty line before the text and another empty line
    #So we will skip the first two lines, get the thrid line and skip the fourth one 
    print("Reading file")
    with open(file_name, 'r') as file:   
        file_data = file.read().splitlines()
    
    if lower:
        file_data = [s.lower() for s in file_data]
        
    return file_data

def get_all_words_no_anotation(data):
    words = []
    for sen in data:
        for word in sen.split(' '):
            words.append(word)
    
    return words

def get_all_words_pos(data):
    pos_list = []
    for sen in data:
        for token in sen.split():
            word_split = token.rsplit('/', 1)
            word_pos = (word_split[0], word_split[1])
            pos_list.append(word_pos)
    
    return pos_list

def get_word_freq_per_pos(data):
    counts = {}
    for word, pos in data:
        if word not in counts:
            counts[word] = {}

        counts[word][pos] = counts[word].get(pos, 0) + 1
        
    return counts

def get_word_most_freq_pos(data):
    return {word: max(data[word], key=data[word].get) for word in data}

def fill_for_missing_word(data):
    all_pos_tags = [pos_tag for pos_dict in data.values() for pos_tag in pos_dict]

    return max(set(all_pos_tags), key=all_pos_tags.count)

def predict(train_pos_dist_data, test_data, fill_pos_dist, word_sample = False, fill_sample = False):
    random.seed(42)
    word_pos_pred = []
    if not word_sample:
        train_word_most_freq_pos = get_word_most_freq_pos(train_pos_dist_data)
    if not fill_sample:
        fill_value = fill_for_missing_word(train_pos_dist_data)
        
    for sentence in test_data:
        for word in sentence.split():
            if word not in train_pos_dist_data:
                if fill_sample:
                    pos = random.choices(list(fill_pos_dist.keys()), weights = list(fill_pos_dist.values()))[0]
                else:
                    pos = fill_value

                word_pos_pred.append(pos)
            else:
                if word_sample:
                    pos = random.choices(list(train_pos_dist_data[word].keys()), weights = list(train_pos_dist_data[word].values()))[0]
                else:
                    pos = train_word_most_freq_pos[word]

                word_pos_pred.append(pos)
    
    return word_pos_pred

def calc_Accuracy(y_pred, y_true):
    if len(y_pred) != len(y_true):
        raise ValueError("Lists need to be the same length")
    
    return sum(p == t for p, t in zip(y_pred, y_true)) / len(y_true)

In [55]:
train_pos_dist_data

NameError: name 'train_pos_dist_data' is not defined

In [24]:
from collections import Counter
import random
import gensim.downloader as dl
import numpy as np


#Returns 2 values: Article dicts: containing the Topic and the article text
def get_file_data(file_name, lower = False):
    #Each topic is separated by header line, empty line before the text and another empty line
    #So we will skip the first two lines, get the thrid line and skip the fourth one 
    print("Reading file")
    with open(file_name, 'r') as file:   
        file_data = file.read().splitlines()
    
    if lower:
        file_data = [s.lower() for s in file_data]
        
    return file_data

def get_all_words_no_anotation(data):
    words = []
    for sen in data:
        for word in sen.split(' '):
            words.append(word)
    
    return words

def get_all_words_pos(data):
    pos_list = []
    for sen in data:
        for token in sen.split():
            word_split = token.rsplit('/', 1)
            word_pos = (word_split[0], word_split[1])
            pos_list.append(word_pos)
    
    return pos_list

def get_word_freq_per_pos(data):
    counts = {}
    for word, pos in data:
        if word not in counts:
            counts[word] = {}

        counts[word][pos] = counts[word].get(pos, 0) + 1
        
    return counts

def get_word_most_freq_pos(data):
    return {word: max(data[word], key=data[word].get) for word in data}

def fill_for_missing_word(data):
    all_pos_tags = [pos_tag for pos_dict in data.values() for pos_tag in pos_dict]

    return max(set(all_pos_tags), key=all_pos_tags.count)

def get_most_similar_words(model, words, n=20):
    """
    returns dict: The top-n most similar words to each one in words, by the given model.
    """
    most_similar_dict = {}
    for w in words:
        most_similar_dict[w] = model.most_similar(w, topn=n)
    return most_similar_dict

def fill_missing_using_word2vec(data, word, k = 5):
    res = None
    k_count = 0
    try:
        for item in model.most_similar(word, 100):
            if item[0] in data:
                res.append(item[0])
                k_count += 1
            if k_count == k:
                break
    except Exception:
        pass 

    return res

def get_top_k_known_similar_words(mdl, word, known_vocab, k=5):
    '''ensuring known words'''
    k_counter = 0
    similar_known_words = []
    try:
        most_similar_words = mdl.most_similar(word, topn=100)
        for similar_word, cos_similar in most_similar_words:
            if (similar_word in known_vocab) and (similar_word.isalpha()) and (len(similar_word)>1):
                k_counter += 1
                similar_known_words.append(similar_word)
            if k_counter == k:
                break
    except Exception:
        pass 
    
    return similar_known_words

def predict(train_pos_dist_data, test_data, fill_pos_dist, word_sample = False, fill_sample = False):
    random.seed(42)
    word_pos_pred = []
    if not word_sample:
        train_word_most_freq_pos = get_word_most_freq_pos(train_pos_dist_data)
    if not fill_sample:
        fill_value = fill_for_missing_word(train_pos_dist_data)
        
    for sentence in test_data:
        for word in sentence.split():
            if word not in train_pos_dist_data:
                if fill_sample:
                    pos = random.choices(list(fill_pos_dist.keys()), weights = list(fill_pos_dist.values()))[0]
                else:
                    pos = fill_value

                word_pos_pred.append(pos)
            else:
                if word_sample:
                    pos = random.choices(list(train_pos_dist_data[word].keys()), weights = list(train_pos_dist_data[word].values()))[0]
                else:
                    pos = train_word_most_freq_pos[word]

                word_pos_pred.append(pos)
    
    return word_pos_pred

def calc_Accuracy(y_pred, y_true):
    if len(y_pred) != len(y_true):
        raise ValueError("Lists need to be the same length")
    
    return sum(p == t for p, t in zip(y_pred, y_true)) / len(y_true)

def predict_with_inflection(train_pos_dist_data, test_data, fill_pos_dist, word_sample = False, fill_sample = False):
    random.seed(42)
    word_pos_pred = []
    if not word_sample:
        train_word_most_freq_pos = get_word_most_freq_pos(train_pos_dist_data)
    if not fill_sample:
        fill_value = fill_for_missing_word(train_pos_dist_data)
        
    for sentence in test_data:
        for word in sentence.split():
            if all(w not in train_pos_dist_data for w in [word, word.lower(), word.capitalize()]):
                if fill_sample:
                    pos = random.choices(list(fill_pos_dist.keys()), weights = list(fill_pos_dist.values()))[0]
                else:
                    pos = fill_value

                word_pos_pred.append(pos)
            else:
                if word in train_pos_dist_data:
                    pass
                elif word.lower() in train_pos_dist_data:
                    word = word.lower()
                elif word.capitalize() in train_pos_dist_data:
                    word = word.capitalize()
                    
                if word_sample:
                    pos = random.choices(list(train_pos_dist_data[word].keys()), weights = list(train_pos_dist_data[word].values()))[0]
                else:
                    pos = train_word_most_freq_pos[word]

                word_pos_pred.append(pos)
    
    return word_pos_pred

def get_bigram_dict(data):
    counts = {}
    for sentence in data:
        prev_pos = ''
        for token in sentence.split():
            word_split = token.rsplit('/', 1)
            pos_word = f'{prev_pos}_{word_split[0]}'
            if pos_word not in counts:
                counts[pos_word] = {}

            counts[pos_word][word_split[1]] = counts[pos_word].get(word_split[1], 0) + 1
            prev_pos = word_split[1]
            
    return counts

def bigram_predict_static(train_pos_word_dist_data, train_pos_dist_data, test_data, fill_pos_dist):
    random.seed(42)
    word_pos_pred = []
    train_pos_word_most_freq_pos = get_word_most_freq_pos(train_pos_word_dist_data)
    train_word_most_freq_pos = get_word_most_freq_pos(train_pos_dist_data)
    fill_most_freq_pos = fill_for_missing_word(train_pos_dist_data)
    for sentence in test_data:
        prev_pos = ''
        for word in sentence.split():
            pos = None
            if f'{prev_pos}_{word}' not in train_pos_word_dist_data:
                if word not in train_pos_dist_data:
                    if word_vectors:
                        similar_word = get_top_k_known_similar_words(model, word, train_pos_dist_data, 5)
                        possible_pos_fills = [train_pos_word_most_freq_pos.get(f'{prev_pos}_{w}', train_word_most_freq_pos.get(w, fill_most_freq_pos)) for w in similar_word]
                        possible_pos_fills = [fill_most_freq_pos] if possible_pos_fills==[] else possible_pos_fills
                        fill_value = max(possible_pos_fills[::-1], key=possible_pos_fills.count)
                        #fill_value = Counter(possible_pos_fills).most_common(1)[0][0]
                        pos = fill_value
                else:
                    pos = train_word_most_freq_pos[word]
            else:
                pos = train_pos_word_most_freq_pos[f'{prev_pos}_{word}']

            word_pos_pred.append(pos)
            prev_pos = pos
    
    return word_pos_pred

def bigram_predict_with_inflection(train_pos_word_dist_data, train_pos_dist_data, test_data, fill_pos_dist, word_sample = False, fill_sample = False):
    random.seed(42)
    word_pos_pred = []
    train_pos_word_most_freq_pos = get_word_most_freq_pos(train_pos_word_dist_data)
    if not word_sample:
        train_word_most_freq_pos = get_word_most_freq_pos(train_pos_dist_data)       
    if not fill_sample:
        fill_value = fill_for_missing_word(train_pos_dist_data)
    
        
    for sentence in test_data:
        prev_pos = ''
        for word in sentence.split():
            if all(w not in train_pos_word_dist_data for w in [f'{prev_pos}_{word}',
                                                          f'{prev_pos}_{word.lower()}',
                                                          f'{prev_pos}_{word.capitalize()}']):
                if all(w not in train_pos_dist_data for w in [word, word.lower(), word.capitalize()]):
                    pos = fill_value
                else:
                    if word in train_pos_dist_data:
                        pass
                    elif word.lower() in train_pos_dist_data:
                        word = word.lower()
                    elif word.capitalize() in train_pos_dist_data:
                        word = word.capitalize()
                    pos = train_word_most_freq_pos[word]
            else:
                if f'{prev_pos}_{word}' in train_pos_word_dist_data:
                    pass
                elif f'{prev_pos}_{word.lower()}' in train_pos_word_dist_data:
                    word = word.lower()
                elif f'{prev_pos}_{word.capitalize()}' in train_pos_word_dist_data:
                    word = word.capitalize()
                pos = train_pos_word_most_freq_pos[f'{prev_pos}_{word}']

            word_pos_pred.append(pos)
            prev_pos = pos
    
    return word_pos_pred

In [29]:
def get_top_k_known_similar_words(mdl, word, known_vocab, k=5):
    '''ensuring known words'''
    k_counter = 0
    similar_known_words = []
    try:
        most_similar_words = mdl.most_similar(word, topn=100)
        for similar_word, cos_similar in most_similar_words:
            if (similar_word in known_vocab) and (similar_word.isalpha()) and (len(similar_word)>1):
                k_counter += 1
                similar_known_words.append(similar_word)
            if k_counter == k:
                break
    except Exception:
        pass 
    
    return similar_known_words

def bigram_predict_static(train_pos_word_dist_data, train_pos_dist_data, test_data):
    random.seed(42)
    word_pos_pred = []
    train_pos_word_most_freq_pos = utils.get_word_most_freq_pos(train_pos_word_dist_data)
    train_word_most_freq_pos = utils.get_word_most_freq_pos(train_pos_dist_data)
    fill_most_freq_pos = utils.fill_for_missing_word(train_pos_dist_data)
    similarity_model = dl.load("glove-wiki-gigaword-300") # dl.load("word2vec-google-news-300")
    for sentence in test_data:
        prev_pos = ''
        for word in sentence.split():
            pos = None
            if f'{prev_pos}_{word}' not in train_pos_word_dist_data:
                if word not in train_pos_dist_data:
                    similar_word = get_top_k_known_similar_words(similarity_model, word, train_pos_dist_data, 5)
                    possible_pos_fills = [train_pos_word_most_freq_pos.get(f'{prev_pos}_{w}', train_word_most_freq_pos.get(w, fill_most_freq_pos)) for w in similar_word]
                    possible_pos_fills = [fill_most_freq_pos] if possible_pos_fills==[] else possible_pos_fills
                    fill_value = max(possible_pos_fills[::-1], key=possible_pos_fills.count)
                    #fill_value = Counter(possible_pos_fills).most_common(1)[0][0]
                    pos = fill_value
                else:
                    pos = train_word_most_freq_pos[word]
            else:
                pos = train_pos_word_most_freq_pos[f'{prev_pos}_{word}']

            word_pos_pred.append(pos)
            prev_pos = pos
    
    return word_pos_pred

In [ ]:
train_file = './data/pos/ass1-tagger-train'
train_sentences = get_file_data(train_file)
words_pos = get_all_words_pos(train_sentences)
train_data_freq = get_word_freq_per_pos(words_pos)
train_data_mdl = get_word_most_freq_pos(train_data_freq)

Reading file


In [54]:
train_sentences

["In/IN an/DT Oct./NNP 19/CD review/NN of/IN ``/`` The/DT Misanthrope/NN ''/'' at/IN Chicago/NNP 's/POS Goodman/NNP Theatre/NNP (/( ``/`` Revitalized/VBN Classics/NNS Take/VBP the/DT Stage/NN in/IN Windy/NNP City/NNP ,/, ''/'' Leisure/NN &/CC Arts/NNS )/) ,/, the/DT role/NN of/IN Celimene/NNP ,/, played/VBN by/IN Kim/NNP Cattrall/NNP ,/, was/VBD mistakenly/RB attributed/VBN to/IN Christina/NNP Haag/NNP ./.",
 'Ms./NNP Haag/NNP plays/VBZ Elianti/NNP ./.',
 'Rolls-Royce/NNP Motor/NNP Cars/NNPS Inc./NNP said/VBD it/PRP expects/VBZ its/PRP$ U.S./NNP sales/NNS to/TO remain/VB steady/JJ at/IN about/IN 1,200/CD cars/NNS in/IN 1990/CD ./.',
 'The/DT luxury/NN auto/NN maker/NN last/JJ year/NN sold/VBD 1,214/CD cars/NNS in/IN the/DT U.S./NNP',
 'Howard/NNP Mosher/NNP ,/, president/NN and/CC chief/JJ executive/NN officer/NN ,/, said/VBD he/PRP anticipates/VBZ growth/NN for/IN the/DT luxury/NN auto/NN maker/NN in/IN Britain/NNP and/CC Europe/NNP ,/, and/CC in/IN Far/JJ Eastern/JJ markets/NNS ./.',

In [12]:
dev_input_file = './data/pos/ass1-tagger-dev-input'
dev_file = './data/pos/ass1-tagger-dev'
dev_sentences = get_file_data(dev_file)
dev_actual = [pos for word, pos in get_all_words_pos(dev_sentences)]
train_pos_word_freq = get_bigram_dict(train_sentences)
dev_input_sentences = get_file_data(dev_input_file)
train_pos_freq_dict = Counter([v for k, v in words_pos])
dev_predict_bigrm = bigram_predict(train_pos_word_freq, train_data_freq, dev_input_sentences, train_pos_freq_dict)
print(calc_Accuracy(dev_predict_bigrm, dev_actual))

Reading file
Reading file


NameError: name 'bigram_predict' is not defined

In [12]:
dev_input_file = './data/pos/ass1-tagger-dev-input'
dev_file = './data/pos/ass1-tagger-dev'
dev_input_sentences = get_file_data(dev_input_file)
dev_sentences = get_file_data(dev_file)
dev_actual = [pos for word, pos in get_all_words_pos(dev_sentences)]
train_pos_freq_dict = Counter([v for k, v in words_pos])
sentences = get_file_data(dev_input_file)
dev_predict_f_f = predict(train_data_freq, dev_input_sentences, train_pos_freq_dict, word_sample = False, fill_sample = False)
dev_predict_f_t = predict(train_data_freq, dev_input_sentences, train_pos_freq_dict, word_sample = False, fill_sample = True)
dev_predict_t_f = predict(train_data_freq, dev_input_sentences, train_pos_freq_dict, word_sample = True, fill_sample = False)
dev_predict_t_t = predict(train_data_freq, dev_input_sentences, train_pos_freq_dict, word_sample = True, fill_sample = True)
print(calc_Accuracy(dev_predict_f_f, dev_actual))
print(calc_Accuracy(dev_predict_f_t, dev_actual))
print(calc_Accuracy(dev_predict_t_f, dev_actual))
print(calc_Accuracy(dev_predict_t_t, dev_actual))


Reading file
Reading file
Reading file
0.911591632176141
0.9042430385449222
0.8902497395613368
0.8836331897401245


In [53]:
train_data_freq

{'In': {'IN': 1735, 'RB': 1, 'NNP': 3, 'RBR': 1},
 'an': {'DT': 3142, ',': 1},
 'Oct.': {'NNP': 317, 'NN': 1},
 '19': {'CD': 100},
 'review': {'NN': 36, 'VB': 21, 'VBP': 1},
 'of': {'IN': 22925, 'RP': 2, 'RB': 2},
 '``': {'``': 6967},
 'The': {'DT': 6795, 'NNP': 37, 'VB': 1},
 'Misanthrope': {'NN': 3},
 "''": {"''": 6787},
 'at': {'IN': 4361, 'RP': 1},
 'Chicago': {'NNP': 197},
 "'s": {'POS': 8079, 'VBZ': 1222, 'PRP': 8, 'NNP': 1, 'NNS': 1},
 'Goodman': {'NNP': 7},
 'Theatre': {'NNP': 5},
 '(': {'(': 1153},
 'Revitalized': {'VBN': 1},
 'Classics': {'NNS': 1},
 'Take': {'VBP': 1, 'VB': 8},
 'the': {'DT': 41098, 'VBP': 1, 'NNP': 5, 'NN': 1, 'JJ': 2},
 'Stage': {'NN': 1, 'NNP': 2},
 'in': {'IN': 14957, 'RP': 173, 'RB': 53, 'FW': 2, 'RBR': 1},
 'Windy': {'NNP': 1},
 'City': {'NNP': 132, 'NN': 7},
 ',': {',': 48723},
 'Leisure': {'NN': 2, 'NNP': 1},
 '&': {'CC': 1033, 'SYM': 1},
 'Arts': {'NNS': 2, 'NNP': 4, 'NNPS': 2},
 ')': {')': 1160},
 'role': {'NN': 125},
 'Celimene': {'NNP': 4},
 'pla

In [15]:
#0.7
 dev_predict_bigrm_1_2 = bigram_predict(train_pos_word_freq, train_data_freq, dev_input_sentences, train_pos_freq_dict, word_vectors = True)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9291606836162964


In [17]:
#0.5
dev_predict_bigrm_1_2 = bigram_predict(train_pos_word_freq, train_data_freq, dev_input_sentences, train_pos_freq_dict, word_vectors = True)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9295267055212997


In [39]:
#0.4
dev_predict_bigrm_1_2 = bigram_predict(train_pos_word_freq, train_data_freq, dev_input_sentences, train_pos_freq_dict, word_vectors = True)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9199819804600614


In [22]:
#0.3
dev_predict_bigrm_1_2 = bigram_predict(train_pos_word_freq, train_data_freq, dev_input_sentences, train_pos_freq_dict, word_vectors = True)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9314694371709322


In [28]:
#0.2
dev_predict_bigrm_1_2 = bigram_predict_static(train_pos_word_freq, train_data_freq, dev_input_sentences)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9327927471351747


In [31]:
#0.2
dev_predict_bigrm_1_2 = bigram_predict_static(train_pos_word_freq, train_data_freq, dev_input_sentences)
print(calc_Accuracy(dev_predict_bigrm_1_2, dev_actual))

0.9327927471351747


In [1]:
import gensim.downloader as dl
model_google = dl.load("word2vec-google-news-300")
model_glove_wiki =  dl.load("glove-wiki-gigaword-300")
model_fastest_wiki =  dl.load("fasttext-wiki-news-subwords-300")


[==================================================] 100.0% 376.1/376.1MB downloaded
[==================================================] 100.0% 958.5/958.4MB downloaded


In [70]:
for k in range(1,6):
    print(k)

1
2
3
4
5


In [56]:
words

['In',
 'an',
 'Oct.',
 '19',
 'review',
 'of',
 '``',
 'The',
 'Misanthrope',
 "''",
 'at',
 'Chicago',
 "'s",
 'Goodman',
 'Theatre',
 '(',
 '``',
 'Revitalized',
 'Classics',
 'Take',
 'the',
 'Stage',
 'in',
 'Windy',
 'City',
 ',',
 "''",
 'Leisure',
 '&',
 'Arts',
 ')',
 ',',
 'the',
 'role',
 'of',
 'Celimene',
 ',',
 'played',
 'by',
 'Kim',
 'Cattrall',
 ',',
 'was',
 'mistakenly',
 'attributed',
 'to',
 'Christina',
 'Haag',
 '.',
 'Ms.',
 'Haag',
 'plays',
 'Elianti',
 '.',
 'Rolls-Royce',
 'Motor',
 'Cars',
 'Inc.',
 'said',
 'it',
 'expects',
 'its',
 'U.S.',
 'sales',
 'to',
 'remain',
 'steady',
 'at',
 'about',
 '1,200',
 'cars',
 'in',
 '1990',
 '.',
 'The',
 'luxury',
 'auto',
 'maker',
 'last',
 'year',
 'sold',
 '1,214',
 'cars',
 'in',
 'the',
 'U.S.',
 'Howard',
 'Mosher',
 ',',
 'president',
 'and',
 'chief',
 'executive',
 'officer',
 ',',
 'said',
 'he',
 'anticipates',
 'growth',
 'for',
 'the',
 'luxury',
 'auto',
 'maker',
 'in',
 'Britain',
 'and',
 'Europe

In [59]:
print(len(list(set(words) - set(model_google.index_to_key))))
print(len(list(set(words) - set(model_glove_wiki.index_to_key))))
print(len(list(set(words) - set(model_fastest_wiki.index_to_key))))

10960
18757
4723


In [3]:
print(len(model.index_to_key))

400000


In [4]:
for mdl_name in list(dl.info()['models'].keys()):
    model = dl.load(mdl_name)
    print(f"Model name: {mdl_name}, {len(model.index_to_key)} {len(list(set(words) - set(model.index_to_key)))}")

NameError: name 'words' is not defined